In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from preprocesamiento_datos import imputar_datos_estacionales, escalar_datos, dividir_train_test
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
import plotly.express as px
import pickle

st.title("Entrenamiento y Predicción con RNN")

uploaded_file = st.file_uploader("Sube el archivo demanda.csv", type="csv")

if uploaded_file is not None:
    df = pd.read_csv(uploaded_file, parse_dates=['datetime'])
    st.success("Datos cargados correctamente.")

    # Preprocesamiento
    df = imputar_datos_estacionales(df)
    df = escalar_datos(df)
    df_train, df_test = dividir_train_test(df)

    # Entrenamiento
    def crear_secuencias(datos, n_pasos):
        X, y = [], []
        for i in range(len(datos) - n_pasos):
            X.append(datos[i:i + n_pasos])
            y.append(datos[i + n_pasos])
        return np.array(X), np.array(y)

    n_pasos = 24
    X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
    X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    if st.button("Entrenar RNN"):
        model = Sequential([
            SimpleRNN(50, activation='tanh', input_shape=(n_pasos, 1)),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')

        with st.spinner("Entrenando modelo..."):
            history = model.fit(X_train, y_train, epochs=20, batch_size=32,
                                validation_data=(X_test, y_test), verbose=0)

        st.success("✅ Modelo entrenado")

        # Mostrar gráfica de pérdida
        df_loss = pd.DataFrame({
            'epoch': range(1, len(history.history['loss']) + 1),
            'train_loss': history.history['loss'],
            'val_loss': history.history['val_loss']
        })
        fig_loss = px.line(df_loss, x='epoch', y=['train_loss', 'val_loss'],
                           labels={'value': 'Loss (MSE)', 'epoch': 'Época'},
                           title='Pérdida del entrenamiento RNN')
        st.plotly_chart(fig_loss, use_container_width=True)

        # Predicción One-Step
        pred_scaled = model.predict(X_test)
        pred = df['scaler'].inverse_transform(pred_scaled)
        y_real = df['scaler'].inverse_transform(y_test.reshape(-1, 1))

        df_pred = pd.DataFrame({
            'Real': y_real.flatten(),
            'Predicción': pred.flatten()
        })

        fig_pred = px.line(df_pred.head(200), title="Predicción vs Real (One-Step)")
        st.plotly_chart(fig_pred, use_container_width=True)

        mse = mean_squared_error(y_real, pred)
        st.metric(label="MSE (Error cuadrático medio)", value=f"{mse:.2f}")
